In [4]:
# Convert groundtruth plys to groundtruth label txts

from src.evaluation_and_conversion import painted_ply_to_label_dict, label_dict_to_label_txt

import os

folder = "../../Data/fumane/groundtruths/"

broken = []
for file in os.listdir(folder+"plys/"):
    print("Convert file ", file)
    file_id = file.split("_c.ply")[0]
    try:
        gt_label_dict = painted_ply_to_label_dict(folder+"plys/"+file)
        label_dict_to_label_txt(gt_label_dict, folder+"txts/"+file_id+"_gt_labels")
    except:
        broken.append(file_id)

print("Broken files:")
print(broken)

Convert file  728_c.ply
painted_ply_to_label_dict took 12.20296 seconds to execute!
label_dict_to_label_txt took 0.07837 seconds to execute!
Convert file  10454_c.ply
painted_ply_to_label_dict took 13.40657 seconds to execute!
label_dict_to_label_txt took 0.08758 seconds to execute!
Convert file  729_c.ply
painted_ply_to_label_dict took 13.00211 seconds to execute!
label_dict_to_label_txt took 0.08124 seconds to execute!
Convert file  4124_c.ply
painted_ply_to_label_dict took 12.76899 seconds to execute!
label_dict_to_label_txt took 0.08253 seconds to execute!
Convert file  2280_c.ply
painted_ply_to_label_dict took 1.56042 seconds to execute!
label_dict_to_label_txt took 0.01020 seconds to execute!
Convert file  6853_c.ply
painted_ply_to_label_dict took 12.24698 seconds to execute!
label_dict_to_label_txt took 0.07250 seconds to execute!
Convert file  5219_c.ply
painted_ply_to_label_dict took 12.32056 seconds to execute!
label_dict_to_label_txt took 0.07429 seconds to execute!
Convert 

In [1]:
# create curvature folder

import os
import shutil

folder = "../../Data/fumane/"

for file in os.listdir(folder+"groundtruths/txts/"):
    file_id = file.split("_")[0]
    # create folder for each id
    dst_path = folder + "curvature/"+file_id+"/"
    os.makedirs(dst_path)
    for curv_file in os.listdir(folder+"data/"):
        if curv_file.split("_")[0] == file_id:
            src_path = folder+"data/"+curv_file
            shutil.copy(src_path, dst_path)

In [3]:
# evaluate

from src.evaluation_and_conversion import painted_ply_to_label_dict, compare_result_txt_to_groundtruth_label_dict

import os
import csv

folder = "../../Data/artefacts/"

with open(folder + "/results/best_results.txt", "w") as fresults:
    fresults.write("ObjectID BestResult BestParams\n")
    
    for dir_name in os.listdir(folder+"results/"):
        obj_id = dir_name.split("_r1.00_n4_v256.volume", 1)[0]
        gt_filepath = folder+"groundtruths/" + obj_id + "_c.ply"
        result_files = []
        
        if dir_name != "best_results.txt":
            for filename in os.listdir(folder+"results/"+dir_name+"/"):

                if filename.endswith(".txt") and not filename.endswith("_correct.txt") and not filename.endswith("_timings.txt"):
                    result_files.append(os.path.join(folder+"results/"+dir_name+"/", filename))
                else:
                    continue

            gt_label = painted_ply_to_label_dict(gt_filepath)

            best_result = 0
            best_param = None

            result_table = []
            #result_table.append(["Persistence", "High_thresh", "Low_thresh", "Merge_thresh", "Correctness"])
            for file in result_files:
                correctness, high_thr, low_thr, merge_thr = compare_result_txt_to_groundtruth_label_dict(file, gt_label, metric="IoU", plot_correctness_mask=False)
                result_table.append([high_thr, low_thr, merge_thr, correctness])
                if correctness > best_result:
                    best_result = correctness
                    best_param = [high_thr, low_thr, merge_thr]

            fresults.write(str(obj_id) + " " + str(best_result) + " " + str(best_param) + "\n")

            with open(folder+"results/"+dir_name+"/"+str(obj_id)+"_result_table.csv", "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerows(result_table)

painted_ply_to_label_dict took 11.98260 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 2.66769 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 2.75364 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 2.21791 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 2.84570 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 2.51176 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 2.91349 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 2.40659 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 2.90353 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 2.19236 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 2.30217 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 2.52476 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 2.20640 seconds to execute!
comp